In [ ]:
import openai
from qdrant_client import QdrantClient

### Embedding function

In [ ]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )
    return response.data[0].embedding

### Retrieval function

In [ ]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [ ]:
def retrieve_data(query, qdrant_client, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    similarity_scores = []
    retrieved_context_ratings = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        retrieved_context_ratings.append(result.payload["average_rating"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "retrieved_context_ratings": retrieved_context_ratings,
        "similarity_scores": similarity_scores,
    }

In [ ]:
retrieved_context = retrieve_data("What kind of earphones can I get?", qdrant_client, k=10)

In [ ]:
retrieved_context

### Format retrieved context function

In [ ]:
def process_context(context):

    formatted_context = ""

    for id, chunk, rating in zip(context["retrieved_context_ids"], context["retrieved_context"], context["retrieved_context_ratings"]):
        formatted_context += f"- ID: {id}, rating: {rating}, description: {chunk}\n"

    return formatted_context

In [ ]:
preprocessed_context = process_context(retrieved_context)

In [ ]:
print(process_context(retrieved_context))

### Create Prompt function

In [ ]:
def build_prompt(preprocessed_context, question):

    prompt = f"""
You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructtions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.

Context:
{preprocessed_context}

Question:
{question}
"""

    return prompt

In [ ]:
prompt = build_prompt(preprocessed_context, "What kind of earphones can I get?")

In [ ]:
print(build_prompt(preprocessed_context, "What kind of earphones can I get?"))

In [ ]:
prompt = build_prompt(preprocessed_context, "What kind of earphones can I get with ratings above 4.5?")

### Generate answer function

In [ ]:
def generate_answer(prompt):

    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[{"role": "system", "content": prompt}],
        temperature=0.5,
    )

    return response.choices[0].message.content

In [ ]:
print(generate_answer(prompt))

### Cimbined RAG Pipeline

In [ ]:
def rag_pipeline(question, top_k=5):

    qdrant_client = QdrantClient(url="http://localhost:6333")

    retrieved_context = retrieve_data(question, qdrant_client, top_k)
    preprocessed_context = process_context(retrieved_context)
    prompt = build_prompt(preprocessed_context, question)
    answer = generate_answer(prompt)

    return answer


In [ ]:
print(rag_pipeline("What kind of earphones can I get with ratings above 4.5?"))